In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
raw_data = pickle.load(open('rawData.pkl', 'rb'))

In [3]:
for channel_id, videos in raw_data.items():
    print(channel_id, videos.keys())

UCgc00bfF_PvO_2AvqJZHXFg dict_keys(['QISBZEq9OXc', 'znoNzgwrLpM', 'lAmuEnePHHo', 'vpozQc7VMVs', '06J5KGmct14', '9N7leomAGUs'])
UCTOSI18KyJSZWouZM5kADvg dict_keys(['R-0S123bEt0', 'VljKRXeBBhI', '8wbssZK5kdc', '0pywwNJu1Is', 'IDW8r6pWKmY', 'GZuXraB0afg', 'SEBk9-KOtgM'])
UCtN63iegUVqBAxdYkZ-UslQ dict_keys(['h2_F5Lzn6SU', 'a0V3Zo2S0HM', 'UyKs_yvSITk', 'DOnmTaYw-_M', 'RO_3Ck2zXnM'])
UCfjnEW3mVXfW0Wb602r1IrQ dict_keys(['kFKdj1Wtrfo', 'Qx0nfUncvCk', 'CmCIis88Va8', 'Y-UYo7bcUyo', 'iuDUNJRzTfQ'])
UCTKDB3h5zMdf_2siDHsd_yw dict_keys(['fDGKTSndxeQ', '3Pxm0kVEW0E', 'f6vrDG3xmDs', 'Z2ff-GIYuRY', 'y-obv6m7MOE', '4b7Pwvj8eCA', 'lnWwM-h4Dpg'])
UCxywry2DJ4iNXINFPqMaYxw dict_keys(['aSDNuT9ore4', 'cyh9PAtgEY8', 'B7_JLUS4LQY', 'u_ZMFS5Xd2A', 'PdMBS6_n0oY'])
UC2vPHIqjFdpPVMa2PGwJuYg dict_keys(['yyziSf6BnsU', '7gDTywG8LPg', 'mVAEXA9M-M4', 'uoaZRxZ9W1A'])
UC_b5Ew_f185LSJ0jMNZ2CAw dict_keys(['CYbRR-mSEfI', '0JY0nySRg40', '8VVv3yfB3P0', 'DmCWuFfRVsk'])
UCmnHY9Uw5pDgn3tvZOtarKw dict_keys(['KVjcuddU2-k', 'xeCIirT

In [4]:
def get_delta(channel, video, k):
    return np.array(
        [ video.__dict__[k] for _,video in raw_data[channel][video].items() ]
    )

In [5]:
def get_mean_std(array):
    return np.mean(array), np.std(array)

In [6]:
def get_log(array):
    w = np.where(array > 0, True, False)
    return np.log10(array, where=w, dtype='float32')

In [11]:
deltas = {}
for channel_key, videos in raw_data.items():
    deltas[channel_key] = {}
    for video_ID, periods in videos.items():
        delta = {}
        delta["like_count"] = get_delta(channel_key, video_ID, 'like_count')
        delta["view_count"] = get_delta(channel_key, video_ID, 'view_count')
        delta["comment_count"] = get_delta(channel_key, video_ID, 'comment_count')
        delta["dislike_count"] = get_delta(channel_key, video_ID, 'dislike_count')
        
        for key in delta.keys():
            delta[key] = get_mean_std(get_log(delta[key]))
        deltas[channel_key][video_ID] = delta

In [26]:
data = list()
for channel_key, videos in deltas.items():
    for video_ID, delta_ in videos.items():
        # todo: Pegar total de inscritos e acrescentar no fim de ROW
#         row = [video_ID, delta_["like_count"], delta_["view_count"], delta_["comment_count"], delta_["dislike_count"]]
        row = [ video_ID ]
        for key in delta_.keys():
            row.extend(delta_[key])
        data.append(row)
        

In [30]:
df = pd.DataFrame(
    data, columns=["video_id", 
                   "likes_mean", "likes_std", 
                   "view_mean", "views_std", 
                   "comment_mean", "comment_std",
                   "dislike_mean", "dislike_std"
                  ]
)

In [33]:
df.to_csv('dados.csv', index=False)